# Estimation Theory - Mini Project 4

Consider the estimation of the covariance of a bivariate Gaussian distribution. We have access to $n$ observations
$y_i \sim \mathcal{N}(0,\Sigma)$. $y_i$ is a $2\times1$ vector, $\Sigma$ is a $2\times2$ matrix. 

$\vec{y}$ is set of all observations $y_i$.

$$\Sigma = \begin{bmatrix} 1 & 0 \\ 0 & 2
\end{bmatrix}$$

and $n = 10, 100, 1000$

In [1]:
from pylab import *

In [2]:
from scipy.stats import invwishart, invgamma, multivariate_normal
from scipy.stats import beta
import scipy.optimize

In [3]:
from tqdm import tqdm_notebook

In [4]:
rcParams['figure.figsize'] = 12,9
rcParams['axes.grid'] = True
rcParams['font.size'] = 18
rcParams['figure.facecolor'] = 'w'

In [5]:
sigma = np.array([[1, 0], [0, 2]])

In [6]:
def getY(n, repeats=100, sigma=None):
    sigma = array([[1.0, 0], [0, 2]]) if sigma is None else sigma
    samples = multivariate_normal.rvs(mean=zeros(len(sigma)), cov=sigma, size=(repeats, n))
    return samples

In [7]:
y = getY(100)

In [8]:
def eval_estimator(estimator, sigma=None, ns=[10, 100, 1000], repeats=1000):
    sigma = array([[1.0, 0], [0, 2]]) if sigma==None else sigma
    mses = []
    for n in ns:
        y = getY(n=n, repeats=repeats, sigma=sigma)
        estimates = estimator(y) # (repeats, 2, 2)
        mse = (norm(estimates - sigma, axis=(1, 2))**2).mean()
        mses.append(mse)
    return mses

# Monte Carlo Bayesian estimation:
This method is useful when the posterior is not available in closed form. Note that we
require the mean of the posterior distribution.

<img src='4.4.1.png'>

The likelihood is,

<img src='4.4.2.png'>

Instead of using the closed form expression for the posterior update, find the posterior using Monte Carlo integration using the following equation

<img src='4.4.3.png'>

where each $\Sigma_j \sim p(\Sigma)$ (a sample drawn from the prior distribution). Report the values of $A$ for $n = 10; 100; 1000$ and for $m = 10^3, 10^4, 10^5$ and for $p(\Sigma) = \sim InvWishart_{\nu_0}(\Delta_0)$ for the following paramters:

(a) $\nu_0 = 5$, $\Delta_0 = \begin{bmatrix} 4 & 0 \\ 0 & 5\end{bmatrix}$

(b) $\nu_0 = 5$, $\Delta_0 = \begin{bmatrix} 2 & 0 \\ 0 & 4\end{bmatrix}$

Which prior performs better? Why do you think it happens? Can you justify why modeling the prior is important? Note that you can  now model your prior distribution as any non-conjugate distribution as well.

In [9]:
v = 5
delta0 = np.array([[4,0], [0,5]])
delta1 = np.array([[2,0], [0,4]])

In [10]:
def monte_carlo(y, v=5, delta=delta0, n=100, m=10000, repeats=100):
    p_sigma = invwishart.rvs(v, delta, size=(repeats, m), random_state=None)
    d = np.power(np.linalg.det(p_sigma), -n/2)
    A = []
    for i in range(repeats):
        expo = np.exp(-0.5*np.einsum('ni,jik,nk->j', y[i], np.linalg.inv(p_sigma[i]), y[i]))
        a_den = np.sum(d[i]*expo)
        a_num = np.sum(np.einsum('nij,n->nij', p_sigma[i], d[i]*expo), axis=0)
        A.append(a_num/a_den)
    return A

<font size=5>$$A = \frac{\sum\limits_{j=1}^m \Big[\Sigma_j \prod_{i=1}^n \frac{1}{|\Sigma_j|^{1/2}} exp\big( -\frac{1}{2}y_i^T\Sigma_j^{-1}y_i \big) \Big]}{\sum\limits_{j=1}^m \Big[ \prod_{i=1}^n \frac{1}{|\Sigma_j|^{1/2}} exp\big( -\frac{1}{2}y_i^T\Sigma_j^{-1}y_i \big) \Big]}$$

In [11]:
def bayes_monte_carlo(y, v=5, delta=delta0, n=100, m=10000, repeats=100):
    p_sigma = invwishart.rvs(v, delta, size=(repeats, m), random_state=None)
    d = np.power(np.linalg.det(p_sigma), -1/2)
    A = []
    for i in range(repeats):
        e = np.exp(-0.5*np.einsum('ni,jik,nk->nj', y[i], np.linalg.inv(p_sigma[i]), y[i]))
        expo = np.einsum('ij,j->ij', e, d[i])
        product = np.prod(expo, axis=0)
        a_den = np.sum(product)
        a_num = np.sum(np.einsum('nij,n->nij', p_sigma[i], product), axis=0)
        A.append(a_num/a_den)
    return A

In [12]:
y = getY(n = 100, repeats=100)

In [13]:
r0 = bayes_monte_carlo(y, delta=delta0, m=1000, repeats=100)
r1 = bayes_monte_carlo(y, delta=delta0, m=10000, repeats=100)
r2 = bayes_monte_carlo(y, delta=delta0, m=100000, repeats=100)

In [14]:
print(np.sum(r0, axis=0)/100)
print(np.sum(r1, axis=0)/100)
print(np.sum(r2, axis=0)/100)

[[1.03961523e+00 6.47337374e-04]
 [6.47337374e-04 1.98565210e+00]]
[[ 1.04299112 -0.00224548]
 [-0.00224548  1.97603585]]
[[ 1.04277443e+00 -2.57396711e-04]
 [-2.57396711e-04  1.97458875e+00]]


In [15]:
eval_estimator(bayes_monte_carlo, sigma=None, ns=[10, 100, 1000], repeats=1000)

C:\Users\Home\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


[0.9921028980357329, 0.1461985002020579, nan]